In [ ]:
#@title Montar el drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title instalacion torch
!pip install torch_dct

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
#@title Utils Restauracion

import numpy as np
import cv2

import torch
import torch_dct as dct
from torchvision.transforms import functional as F

def flip_odd_lines(matrix):
    """
    Flip odd lines of a matrix
    """
    matrix = matrix.clone()

    matrix[..., 1::2, :] = matrix[..., 1::2, :].flip(-1)

    return matrix

rotate = lambda m, r: torch.rot90(m, r, [-2, -1])
sequency_vec = lambda m: flip_odd_lines(rotate(m, 0)).flatten(start_dim= m.dim()-2)
sequency_mat = lambda v, s: rotate(flip_odd_lines(v.unflatten(-1, s)), 0)


def modulo(x, L):
    positive = x > 0
    x = x % L
    x = torch.where( ( x == 0) &  positive, L, x)
    return x


def center_modulo(x, L):
    return modulo(x + L/2, L) - L/2


def unmodulo(psi):

    psi = torch.nn.functional.pad(psi, (1,1), mode='constant', value=0)
    psi = torch.diff(psi, 1)
    psi = dct.dct(psi, norm='ortho')
    N = psi.shape[-1]
    k = torch.arange(0, N)
    denom = 2*( torch.cos(  torch.pi * k / N  )  -  1  )
    denom[0] = 1.0
    denom = denom.unsqueeze(0).unsqueeze(0) + 1e-7
    psi     = psi / denom
    psi[..., 0] = 0.0
    psi = dct.idct(psi, norm='ortho')
    return psi

RD = lambda x, L: torch.round( x / L)  * L


def hard_thresholding(x, t):
    return x * (torch.abs(x) > t)


def stripe_estimation(psi, t=0.15):

    dx = torch.diff(psi, 1, dim=-1)
    dy = torch.diff(psi, 1, dim=-2)

    dx = hard_thresholding(dx, t)
    dy = hard_thresholding(dy, t)

    dx = F.pad(dx, (1, 0, 1, 0))
    dy = F.pad(dy, (0, 1, 0, 1))


    rho = torch.diff(dx, 1, dim=-1) + torch.diff(dy, 1, dim=-2)
    dct_rho = dct.dct_2d(rho, norm='ortho')


    MX = rho.shape[-2]
    NX = rho.shape[-1]

    I, J = torch.meshgrid(torch.arange(0, MX), torch.arange(0, NX), indexing="ij")
    I = I.to(rho.device)
    J = J.to(rho.device)
    denom = 2 * (torch.cos(torch.pi * I / MX ) + torch.cos(torch.pi * J / NX ) - 2)
    denom = denom.unsqueeze(0).unsqueeze(0)
    denom = denom.to(rho.device)
    dct_phi = dct_rho / denom
    dct_phi[..., 0, 0] = 0
    phi = dct.idct_2d(dct_phi, norm='ortho')
    phi = phi - torch.min(phi)
    # phi = phi - torch.amin(phi, dim=(-1, -2), keepdim=True)
    # phi = RD(phi, 1.0)
    return phi


def recons(m_t, DO=1, L=1.0, vertical=False, t=0.6):

    if vertical:
        m_t = m_t.permute(0, 1, 3, 2)

    shape = m_t.shape[-2:]

    modulo_vec = sequency_vec(m_t)
    res = center_modulo( torch.diff(modulo_vec, n=DO), L) - torch.diff(modulo_vec, n=DO)
    bl = res

    for i in range(DO):
        bl = unmodulo(bl)
        bl = RD(bl, L)

    x_est = bl

    x_est = sequency_mat(x_est, shape)
    x_est = x_est + m_t

    if vertical:
        x_est = x_est.permute(0, 1, 3, 2)

    stripes = stripe_estimation(x_est, t=t)
    x_est2 = x_est  - stripes

    return x_est2

In [ ]:
#@title Procesamiento completo
import os
import cv2
import numpy as np
from tqdm import tqdm

def modulo_fn(x):
    return x - np.floor(x)

# Directorios de entrada y salida
input_folder = "/content/drive/MyDrive/Dataset_standar_procesado/images_base"
output_folder_modulo = "/content/drive/MyDrive/Dataset_modulo/images_2"
output_folder_recovery = "/content/drive/MyDrive/dataset_restaured/images_2"


# Obtener la lista de imágenes en la carpeta
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

alpha = 2
offset = 0.01
k = 5
ksize = (k, k)

# Procesar cada imagen
for img_name in tqdm(image_files, desc="Procesando imágenes"):

    # Leer imagen
    img_path = os.path.join(input_folder, img_name)

    ######################### MODULO #########################
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img  , cv2.COLOR_BGR2RGB).astype(np.float32)  / 256.0
    img = cv2.blur(img, (5, 5))
    img = (img + offset) / (1 + offset)
    imagen_modulo = modulo_fn(img * alpha)
    imagen_modulo_cp = imagen_modulo.copy()

    #Guardar imagen modulo
    output_path_modulo = os.path.join(output_folder_modulo, img_name)
    cv2.imwrite(output_path_modulo, cv2.cvtColor(imagen_modulo * 255, cv2.COLOR_RGB2BGR))

    ######################### RECOVERY #########################
    img = imagen_modulo_cp
    img_t = torch.from_numpy(img).unsqueeze(0).permute(0, 3, 1, 2)
    img_restaured = recons(img_t, DO=1, vertical=True).permute(0, 2, 3, 1)
    img_restaured = img_restaured.squeeze(0).cpu().numpy()
    img_restaured = img_restaured - img_restaured.min()
    img_restaured = img_restaured / img_restaured.max()

    img_restaured = img_restaured - img_restaured.min(axis=(0, 1), keepdims=True)
    img_restaured = img_restaured / img_restaured.max(axis=(0, 1), keepdims=True)

    #Guardar recovery
    output_path_recovery = os.path.join(output_folder_recovery, img_name)
    cv2.imwrite(output_path_recovery, cv2.cvtColor(img_restaured* 255, cv2.COLOR_RGB2BGR))


print("✅ Transformación completada. Imágenes MODULO guardadas en", output_folder_modulo)
print("✅ Transformación completada. Imágenes RECOVERY guardadas en", output_folder_recovery)


Procesando imágenes: 100%|██████████| 1359/1359 [16:00<00:00,  1.41it/s]

✅ Transformación completada. Imágenes MODULO guardadas en /content/drive/MyDrive/Dataset_modulo/images_2
✅ Transformación completada. Imágenes RECOVERY guardadas en /content/drive/MyDrive/dataset_restaured/images_2
